In [1]:
# Imports 
import numpy as np 
import pandas as pd 
import requests 
import math 
from scipy import stats 
import xlsxwriter

In [2]:
# Importing Stocks 
stocks = pd.read_csv('sp_500_stocks.csv')

In [3]:
# API Token 
from secrets import IEX_CLOUD_API_TOKEN

In [5]:
# Testing API Call 
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
print(data)


{'companyName': 'Apple Inc', 'marketcap': 2305137872920, 'week52high': 149.46, 'week52low': 57.31, 'week52change': 0.739552432932713, 'sharesOutstanding': 17302957467, 'float': 0, 'avg10Volume': 122493472, 'avg30Volume': 111945451, 'day200MovingAvg': 124.13, 'day50MovingAvg': 136.8, 'employees': 0, 'ttmEPS': 3.83, 'ttmDividendRate': 0.826559743548432, 'dividendYield': 0.005965371943616458, 'nextDividendDate': '0', 'exDividendDate': '2021-01-27', 'nextEarningsDate': '2021-01-24', 'peRatio': 37.49680318834053, 'beta': 1.1714672784461062, 'maxChangePercent': 52.22531780636636, 'year5ChangePercent': 5.20948209326802, 'year2ChangePercent': 2.2891363163731917, 'year1ChangePercent': 0.7661243865028832, 'ytdChangePercent': 0.03670093027846998, 'month6ChangePercent': 0.2665774276539712, 'month3ChangePercent': 0.20687015077495943, 'month1ChangePercent': 0.06460783337094152, 'day30ChangePercent': 0.05096351702186215, 'day5ChangePercent': 0.002255766844361635}


In [6]:
# Batch API Call Function 
def chunks(lst, n): 
    for i in range(0, len(lst), n): 
        yield lst[i:i + n] 

In [8]:
# Batch API Call 
columns = ['Ticker', 'Stock Price', 'One-Year Price Return', 'Number of Shares to Buy']

symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)): 
    symbol_strings.append(','.join(symbol_groups[i]))

In [50]:
final_dataframe = pd.DataFrame(columns=columns)

for symbol_string in symbol_strings: 
    batch_api_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=price,stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
            pd.Series(
                [
                    symbol, 
                    data[symbol]['price'],
                    data[symbol]['stats']['year1ChangePercent'],
                    'N/A'                
                ], 
                index = columns
            ),
            ignore_index=True
        )
final_dataframe

,Ticker,Stock Price,One-Year Price Return,Number of Shares to Buy
0,A,123.700,0.514962,N/A
1,AAL,17.860,-0.407278,N/A
2,AAP,162.061,0.217644,N/A
3,AAPL,139.890,0.741786,N/A
4,ABBV,112.030,0.35983,N/A
...,...,...,...,...
500,YUM,109.290,-0.0168054,N/A
501,ZBH,162.017,0.0350213,N/A
502,ZBRA,412.130,0.667953,N/A
503,ZION,48.330,0.0734333,N/A


In [27]:
final_dataframe.sort_values('One-Year Price Return', ascending=False, inplace=True)
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(inplace=True)
final_dataframe

,level_0,index,Ticker,Stock Price,One-Year Price Return,Number of Shares to Buy
0,0,78,CARR,40.560,2.2178,N/A
1,1,179,FCX,32.984,1.49312,N/A
2,2,24,ALGN,638.670,1.42339,N/A
3,3,387,PYPL,273.730,1.25593,N/A
4,4,345,NVDA,547.500,1.23901,N/A
5,5,275,LB,50.380,1.05825,N/A
6,6,410,SIVB,514.580,0.95784,N/A
7,7,385,PWR,75.820,0.948014,N/A
8,8,128,DE,312.590,0.930803,N/A
9,9,85,CDNS,142.190,0.880244,N/A


In [29]:
# Calculating Number of Shares to Buy 
def portfolio_input(): 
    global portfolio_size
    portfolio_size = input('Enter the Size of your Portfolio')

    try: 
        float(portfolio_size)
    except ValueError: 
        print('Please enter an integer')
        portfolio_size = input('Enter the Value of Portfolio: ')
        portfolio_size = float(portfolio_size)

In [32]:
portfolio_input()

In [36]:
position_size = float(portfolio_size) / len(final_dataframe.index)

for i in range(0, len(final_dataframe.index)): 
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / final_dataframe['Stock Price'][i])
final_dataframe

,level_0,index,Ticker,Stock Price,One-Year Price Return,Number of Shares to Buy
0,0,78,CARR,40.560,2.2178,4930
1,1,179,FCX,32.984,1.49312,6063
2,2,24,ALGN,638.670,1.42339,313
3,3,387,PYPL,273.730,1.25593,730
4,4,345,NVDA,547.500,1.23901,365
5,5,275,LB,50.380,1.05825,3969
6,6,410,SIVB,514.580,0.95784,388
7,7,385,PWR,75.820,0.948014,2637
8,8,128,DE,312.590,0.930803,639
9,9,85,CDNS,142.190,0.880244,1406


In [93]:
# Differentiating Between High and Low Quality Momentum
hqm_columns = [
    'Ticker', 
    'Stock Price', 
    'Number of Shares to Buy',
    'One Year Price Return',
    'One Year Return Percentile',
    'Six Month Price Return', 
    'Six Month Return Percentile', 
    'Three Month Price Return', 
    'Three Month Return Percentile', 
    'One Month Price Return',
    'One Month Return Percentile']

hqm_dataframe = pd.DataFrame(columns = hqm_columns)

In [94]:
# Batch API Call 
for symbol_string in symbol_strings: 
    batch_api_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=price,stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_url).json()
    for symbol in symbol_string.split(','): 
        hqm_dataframe = hqm_dataframe.append(
            pd.Series(
                [
                    symbol, 
                    data[symbol]['price'],
                    'N/A',
                    data[symbol]['stats']['year1ChangePercent'],
                    'N/A',
                    data[symbol]['stats']['month6ChangePercent'], 
                    'N/A', 
                    data[symbol]['stats']['month3ChangePercent'], 
                    'N/A', 
                    data[symbol]['stats']['month1ChangePercent'],
                    'N/A'
                ], 
                index = hqm_columns
            ),
            ignore_index = True
        )

In [96]:
# Momentum Percentiles 
time_periods = [
    'One Year',
    'Six Month', 
    'Three Month', 
    'One Month'
]

for row in hqm_dataframe.index:
    for time_period in time_periods:
        if hqm_dataframe.loc[row, f'{time_period} Price Return'] == None:
            hqm_dataframe.loc[row, f'{time_period} Price Return'] = 0

for row in hqm_dataframe.index: 
    for time_period in time_periods: 
        hqm_dataframe.loc[row, f'{time_period} Return Percentile'] = stats.percentileofscore(hqm_dataframe[f'{time_period} Price Return'],
        hqm_dataframe.loc[row, f'{time_period} Price Return'])/100
hqm_dataframe

,Ticker,Stock Price,Number of Shares to Buy,One Year Price Return,One Year Return Percentile,Six Month Price Return,Six Month Return Percentile,Three Month Price Return,Three Month Return Percentile,One Month Price Return,One Month Return Percentile
0,A,127.740,N/A,0.497747,0.889109,0.281931,0.625743,0.148188,0.512871,0.0471176,0.528713
1,AAL,17.632,N/A,-0.405869,0.0138614,0.529475,0.875248,0.585567,0.948515,0.15182,0.90099
2,AAP,160.938,N/A,0.220979,0.661386,0.0423563,0.219802,0.0212104,0.237624,0.00751205,0.332673
3,AAPL,140.227,N/A,0.745779,0.970297,0.262147,0.6,0.206732,0.617822,0.0622833,0.623762
4,ABBV,109.160,N/A,0.370771,0.817822,0.184003,0.475248,0.163337,0.538614,0.0436816,0.506931
...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,106.100,N/A,-0.0164015,0.29703,0.136122,0.378218,0.0495164,0.29703,-0.0274392,0.146535
501,ZBH,165.700,N/A,0.0343683,0.380198,0.244199,0.574257,0.18244,0.574257,0.0512133,0.552475
502,ZBRA,414.780,N/A,0.650366,0.938614,0.450341,0.817822,0.237538,0.663366,0.0807401,0.691089
503,ZION,47.719,N/A,0.0728769,0.433663,0.538644,0.877228,0.578995,0.944554,0.129096,0.849505


In [101]:
# Calculating HQM Score
from statistics import mean

for row in hqm_dataframe.index: 
    momentum_pct = []
    for time_period in time_periods: 
        momentum_pct.append(hqm_dataframe.loc[row, f'{time_period} Return Percentile'])
    hqm_dataframe.loc[row, 'HQM Score'] = mean(momentum_pct)

hqm_dataframe

,Ticker,Stock Price,Number of Shares to Buy,One Year Price Return,One Year Return Percentile,Six Month Price Return,Six Month Return Percentile,Three Month Price Return,Three Month Return Percentile,One Month Price Return,One Month Return Percentile,HQM Score
0,A,127.740,N/A,0.497747,0.889109,0.281931,0.625743,0.148188,0.512871,0.0471176,0.528713,0.639109
1,AAL,17.632,N/A,-0.405869,0.0138614,0.529475,0.875248,0.585567,0.948515,0.15182,0.90099,0.684653
2,AAP,160.938,N/A,0.220979,0.661386,0.0423563,0.219802,0.0212104,0.237624,0.00751205,0.332673,0.362871
3,AAPL,140.227,N/A,0.745779,0.970297,0.262147,0.6,0.206732,0.617822,0.0622833,0.623762,0.702970
4,ABBV,109.160,N/A,0.370771,0.817822,0.184003,0.475248,0.163337,0.538614,0.0436816,0.506931,0.584653
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,106.100,N/A,-0.0164015,0.29703,0.136122,0.378218,0.0495164,0.29703,-0.0274392,0.146535,0.279703
501,ZBH,165.700,N/A,0.0343683,0.380198,0.244199,0.574257,0.18244,0.574257,0.0512133,0.552475,0.520297
502,ZBRA,414.780,N/A,0.650366,0.938614,0.450341,0.817822,0.237538,0.663366,0.0807401,0.691089,0.777723
503,ZION,47.719,N/A,0.0728769,0.433663,0.538644,0.877228,0.578995,0.944554,0.129096,0.849505,0.776238


In [111]:
hqm_dataframe.sort_values('HQM Score', ascending=False, inplace=True)
hqm_dataframe = hqm_dataframe[:50]
hqm_dataframe.reset_index(inplace=True)

,level_0,Ticker,Stock Price,Number of Shares to Buy,One Year Price Return,One Year Return Percentile,Six Month Price Return,Six Month Return Percentile,Three Month Price Return,Three Month Return Percentile,One Month Price Return,One Month Return Percentile,HQM Score
0,0,SIVB,498.400,N/A,0.951877,0.988119,1.19859,0.992079,0.68234,0.976238,0.268176,0.980198,0.984158
1,1,VIAC,54.410,N/A,0.617778,0.924752,1.01184,0.984158,0.771495,0.980198,0.425645,1,0.972277
2,2,GM,56.588,N/A,0.601533,0.920792,1.147,0.990099,0.555424,0.934653,0.341131,0.990099,0.958911
3,3,LB,51.440,N/A,1.08483,0.990099,0.946302,0.976238,0.428456,0.857426,0.222754,0.976238,0.950000
4,4,DISCA,43.850,N/A,0.38924,0.829703,0.895456,0.972277,1.04013,0.992079,0.348262,0.99604,0.947525
5,5,AMAT,104.454,N/A,0.685541,0.950495,0.60455,0.906931,0.608637,0.962376,0.190527,0.952475,0.943069
6,6,FCX,31.908,N/A,1.5124,0.99802,1.3305,0.99604,0.66955,0.974257,0.101766,0.764356,0.933168
7,7,IVZ,22.230,N/A,0.308402,0.770297,1.31215,0.994059,0.602836,0.956436,0.291307,0.986139,0.926733
8,8,ALGN,628.850,N/A,1.42369,0.99604,1.14368,0.988119,0.281066,0.712871,0.171275,0.940594,0.909406
9,9,MOS,27.610,N/A,0.341792,0.792079,0.778306,0.950495,0.647879,0.970297,0.161781,0.922772,0.908911


In [112]:
hqm_dataframe = hqm_dataframe.drop('level_0', axis = 1)
hqm_dataframe

,Ticker,Stock Price,Number of Shares to Buy,One Year Price Return,One Year Return Percentile,Six Month Price Return,Six Month Return Percentile,Three Month Price Return,Three Month Return Percentile,One Month Price Return,One Month Return Percentile,HQM Score
0,SIVB,498.400,N/A,0.951877,0.988119,1.19859,0.992079,0.68234,0.976238,0.268176,0.980198,0.984158
1,VIAC,54.410,N/A,0.617778,0.924752,1.01184,0.984158,0.771495,0.980198,0.425645,1,0.972277
2,GM,56.588,N/A,0.601533,0.920792,1.147,0.990099,0.555424,0.934653,0.341131,0.990099,0.958911
3,LB,51.440,N/A,1.08483,0.990099,0.946302,0.976238,0.428456,0.857426,0.222754,0.976238,0.950000
4,DISCA,43.850,N/A,0.38924,0.829703,0.895456,0.972277,1.04013,0.992079,0.348262,0.99604,0.947525
5,AMAT,104.454,N/A,0.685541,0.950495,0.60455,0.906931,0.608637,0.962376,0.190527,0.952475,0.943069
6,FCX,31.908,N/A,1.5124,0.99802,1.3305,0.99604,0.66955,0.974257,0.101766,0.764356,0.933168
7,IVZ,22.230,N/A,0.308402,0.770297,1.31215,0.994059,0.602836,0.956436,0.291307,0.986139,0.926733
8,ALGN,628.850,N/A,1.42369,0.99604,1.14368,0.988119,0.281066,0.712871,0.171275,0.940594,0.909406
9,MOS,27.610,N/A,0.341792,0.792079,0.778306,0.950495,0.647879,0.970297,0.161781,0.922772,0.908911


In [121]:
# Number of shares to buy
portfolio_input()

position_size = float(portfolio_size) / len(hqm_dataframe.index)
for i in range(0, len(hqm_dataframe['Ticker'])):
    hqm_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / hqm_dataframe['Stock Price'][i])
hqm_dataframe

,Ticker,Stock Price,Number of Shares to Buy,One Year Price Return,One Year Return Percentile,Six Month Price Return,Six Month Return Percentile,Three Month Price Return,Three Month Return Percentile,One Month Price Return,One Month Return Percentile,HQM Score
0,SIVB,498.400,40,0.951877,0.988119,1.19859,0.992079,0.68234,0.976238,0.268176,0.980198,0.984158
1,VIAC,54.410,367,0.617778,0.924752,1.01184,0.984158,0.771495,0.980198,0.425645,1,0.972277
2,GM,56.588,353,0.601533,0.920792,1.147,0.990099,0.555424,0.934653,0.341131,0.990099,0.958911
3,LB,51.440,388,1.08483,0.990099,0.946302,0.976238,0.428456,0.857426,0.222754,0.976238,0.950000
4,DISCA,43.850,456,0.38924,0.829703,0.895456,0.972277,1.04013,0.992079,0.348262,0.99604,0.947525
5,AMAT,104.454,191,0.685541,0.950495,0.60455,0.906931,0.608637,0.962376,0.190527,0.952475,0.943069
6,FCX,31.908,626,1.5124,0.99802,1.3305,0.99604,0.66955,0.974257,0.101766,0.764356,0.933168
7,IVZ,22.230,899,0.308402,0.770297,1.31215,0.994059,0.602836,0.956436,0.291307,0.986139,0.926733
8,ALGN,628.850,31,1.42369,0.99604,1.14368,0.988119,0.281066,0.712871,0.171275,0.940594,0.909406
9,MOS,27.610,724,0.341792,0.792079,0.778306,0.950495,0.647879,0.970297,0.161781,0.922772,0.908911


In [122]:
writer = pd.ExcelWriter('momentum_strategy.xlsx')
hqm_dataframe.to_excel(writer, sheet_name='Momentum Strategy', index = False)

In [123]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [124]:
column_formats = { 
                    'A': ['Ticker', string_template],
                    'B': ['Price', dollar_template],
                    'C': ['Number of Shares to Buy', integer_template],
                    'D': ['One-Year Price Return', percent_template],
                    'E': ['One-Year Return Percentile', percent_template],
                    'F': ['Six-Month Price Return', percent_template],
                    'G': ['Six-Month Return Percentile', percent_template],
                    'H': ['Three-Month Price Return', percent_template],
                    'I': ['Three-Month Return Percentile', percent_template],
                    'J': ['One-Month Price Return', percent_template],
                    'K': ['One-Month Return Percentile', percent_template],
                    'L': ['HQM Score', integer_template]
                    }

for column in column_formats.keys():
    writer.sheets['Momentum Strategy'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Momentum Strategy'].write(f'{column}1', column_formats[column][0], string_template)

In [125]:
writer.save()